Move data from dataframe to BigQuery

Dataframe already populated with data from API in prev stage

In [1]:
# imports
import os
import pandas as pd
from google.cloud import bigquery



In [2]:
# global params
GCP_PROJECT = os.environ.get('GCP_PROJECT')
BQ_DATASET = os.environ.get('BQ_DATASET')

In [3]:
# get data from local csv (created by Alex is separate notebook)
artworks_df = pd.read_csv('artsy-fartsci-images.csv')

In [4]:
# have a look at the data
artworks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22178 entries, 0 to 22177
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              22178 non-null  int64  
 1   artwork_id              22178 non-null  object 
 2   title                   22178 non-null  object 
 3   category                22177 non-null  object 
 4   medium                  22117 non-null  object 
 5   date                    17143 non-null  object 
 6   height_cm               10731 non-null  float64
 7   width_cm                10701 non-null  float64
 8   collecting_institution  22005 non-null  object 
 9   image_url               22178 non-null  object 
 10  image_url_is_template   22178 non-null  bool   
 11  normalized              22178 non-null  bool   
dtypes: bool(2), float64(2), int64(1), object(7)
memory usage: 1.7+ MB


In [7]:
# rename url col with template
artworks_df.rename(columns={'image_url':'image_url_template'},inplace=True)


In [8]:
# drop is template col - all are true so is redundant
artworks_df.drop(columns=['image_url_is_template','Unnamed: 0'],inplace=True)

In [9]:
# drop normalized col - all are true so redundant
artworks_df.drop(columns=['normalized'],inplace=True)

In [10]:
# create new url col with normalized url
artworks_df['image_url_normalized'] = artworks_df['image_url_template'].apply(lambda x: x.replace('{image_version}','normalized'))



In [12]:
artworks_df.head()

,artwork_id,title,category,medium,date,height_cm,width_cm,collecting_institution,image_url_template,image_url_normalized
0,4d8b92eb4eb68a1b2c000968,Der Kuss (The Kiss),Painting,Oil and gold leaf on canvas,1907-1908,180.1,180.1,"Österreichische Galerie Belvedere, Vienna",https://d32dm0rphc51dk.cloudfront.net/NOpIAwQa...,https://d32dm0rphc51dk.cloudfront.net/NOpIAwQa...
1,4d8b92ee4eb68a1b2c0009ab,The Third of May,Painting,Oil on canvas,1814,266.2,345.2,"Museo Nacional del Prado, Madrid",https://d32dm0rphc51dk.cloudfront.net/m4X41Fun...,https://d32dm0rphc51dk.cloudfront.net/m4X41Fun...
2,4d8b93394eb68a1b2c0010fa,The Company of Frans Banning Cocq and Willem v...,Painting,Oil on canvas,1642,363.2,437.4,"Rijksmuseum, Amsterdam",https://d32dm0rphc51dk.cloudfront.net/IG8ZLvVm...,https://d32dm0rphc51dk.cloudfront.net/IG8ZLvVm...
3,4d8b937c4eb68a1b2c001722,Mona Lisa,Painting,Oil on poplar,ca. 1503-1506,77.0,53.1,Musée du Louvre,https://d32dm0rphc51dk.cloudfront.net/5L1xjKC_...,https://d32dm0rphc51dk.cloudfront.net/5L1xjKC_...
4,4d8b93b04eb68a1b2c001b9d,Luncheon on the Grass (Le Déjeuner sur l'herbe),Painting,Oil on canvas,1863,208.3,264.2,"Musée d'Orsay, Paris",https://d32dm0rphc51dk.cloudfront.net/zFA7cwdk...,https://d32dm0rphc51dk.cloudfront.net/zFA7cwdk...


In [13]:
# load into BigQuery
# get full table name
full_table_name = f'{GCP_PROJECT}.{BQ_DATASET}.image_data'
#source data
source_data = artworks_df
# instantiate client
client = bigquery.Client(project=GCP_PROJECT)
# truncate or append?
write_mode = 'WRITE_TRUNCATE'
# job_config
job_config = bigquery.LoadJobConfig(write_disposition=write_mode)
# job
job = client.load_table_from_dataframe(source_data,full_table_name,job_config=job_config)
# result
result = job.result()